# **Работа с API**

Многие крупные сайты предоставляют доступ к так называемым **API** (англ. Application Programming Interface, рус. Интерфейс Прикладного Программирования). 
 
**API** — это специальные разделы сайта, где информацию можно получать **без разметки**, а **формат запросов и ответов зафиксирован**. API созданы для того, чтобы облегчить взаимодействие с сайтом для сторонних разработчиков.

Рассмотрим на примере социальной сети ВКонтакте особенности API, характерные для более крупных сайтов.

##### **КЛЮЧ АВТОРИЗАЦИИ**
***
Для того чтобы начать работать с API, обычно необходимо получить сервисный ключ авторизации — **токен**.

**Токен** — это средство *идентификации* пользователя или отдельного сеанса работы *в компьютерных сетях и приложениях*. Различают *программные* и *аппаратные* токены.  
Мы будем использовать программный токен, который обычно представляет собой зашифрованную последовательность символов, позволяющую точно идентифицировать объект и определить уровень его привилегий. Он генерируется системой авторизации и привязывается к конкретному сеансу работы, клиенту сети или пакету данных.

Сервисный токен для API ВКонтакте для нашей задачи создаётся вместе с новым приложением. Приложение мы делать, конечно, не будем. Оно нужно только для получения токена, чтобы сделать необходимые выгрузки.

##### **ЗАПРОСЫ ИЗ БРАУЗЕРА**
***
Сначала рассмотрим работу API на простом примере, на основе которого работают многие системы.

Сделаем наш первый запрос из браузера.

Перейдите по следующей ниже ссылке в браузере, подставив вместо слова *TOKEN* ваш персональный сервисный ключ доступа (токен), полученный на предыдущем шаге:

**https://api.vk.com/method/users.get?user_id=1&v=5.95&access_token=TOKEN**

Итак, мы сделали GET-запрос к API ВКонтакте, который состоит из следующих элементов:

* https://api.vk.com/method — домен и URL запроса API; обычно не меняется;
* users.get — название метода, который отдаёт определённый отчёт, в нашем случае это метод для получения информации о пользователе;
* user_id и v — параметры запроса: идентификатор пользователя, о котором хотим получить информацию (в нашем примере мы запрашиваем информацию о первом пользователе), и номер версии API;
* token — токен, который выдаётся только пользователям, имеющим право просматривать определённые данные, например показания счётчиков Яндекс.Метрики вашего проекта; на все остальные запросы без корректного токена система отвечает отказом.

Если мы обратимся к [**документации метода users.get**](https://vk.com/dev/users.get), то увидим, что в ней описано множество других параметров, которые можно получить о пользователе (дата рождения, пол, родной город и другие) — словом, всё то, что мы видим на странице пользователя в интерфейсе или приложении ВКонтакте (конечно, если пользователь их указал).

Добавим к запросу дату рождения и пол (согласно документации, эти параметры надо перечислять в поле fields):

**https://api.vk.com/method/users.get?user_id=1&v=5.95&fields=sex,bdate&access_token=TOKEN**

##### **ЗАПРОС К API ИЗ КОДА**
***
Продолжаем пользоваться всё той же библиотекой requests.

In [2]:
import requests
from pprint import pprint

In [1]:
token = 'dfc65ed2dfc65ed2dfc65ed21fdfba1207ddfc6dfc65ed2bda5c5957a7f5bc7684658c4' # Указываем свой сервисный токен
url = 'https://api.vk.com/method/users.get' # Указываем адрес страницы к которой делаем запрос
params = {'user_id': 1, 'v': 5.95, 'fields': 'sex,bdate', 'access_token': token, 'lang': 'ru'} # Перечисляем параметры нашего запроса в словаре params
response = requests.get(url, params=params) # Отправляем запрос
print(response.text) # Выводим текст ответа на экран

{"response":[{"id":1,"first_name":"Павел","last_name":"Дуров","can_access_closed":true,"is_closed":false,"sex":2,"bdate":"10.10.1984"}]}


Мы получили строку в JSON-формате, которую можно преобразовать в словарь с помощью метода **json()**, после чего можно с лёгкостью обращаться к различным полям.

Словари нагляднее выводить с помощью функции **pprint()**, которую мы уже использовали ранее:

In [3]:
pprint(response.json()) # Выводим содержимое словаря, содержащего ответ, на экран

{'response': [{'bdate': '10.10.1984',
               'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров',
               'sex': 2}]}


По ключу response мы можем получить список, в котором хранятся словари, содержащие информацию о запрошенных нами пользователях. Мы запросили информацию лишь об одном из них, поэтому список содержит только один элемент. Извлечём его:

In [4]:
user = response.json()['response'][0] # Извлекаем из словаря по ключу response информацию о первом пользователе
print(user['bdate']) # Выводим дату рождения первого пользователя на экран

10.10.1984


Метод **users.get()** позволяет запрашивать информацию о множестве (до 1 000) пользователей одновременно. Для этого нужно использовать параметр **user_ids** и передавать id **через запятую в строковом формате**. Например, чтобы получить информацию о пользователях с id=1, id=2, id=3, необходимо передать значение параметра user_ids='1,2,3'.

In [5]:
# Формируем строку, содержащую информацию о поле id первых трёх пользователей
ids = ",".join(map(str, range(1, 4)))
# Формируем строку параметров
params = {'user_ids': ids, 'v': 5.95, 'fields': 'bday', 'access_token': token, 'lang': 'ru'}
# Посылаем запрос, полученный ответ в формате JSON-строки преобразуем в словарь
# и выводим на экран его содержимое, используя функцию pprint()
pprint(requests.get(url, params=params).json())

{'response': [{'can_access_closed': True,
               'first_name': 'Павел',
               'id': 1,
               'is_closed': False,
               'last_name': 'Дуров'},
              {'can_access_closed': False,
               'first_name': 'Александра',
               'id': 2,
               'is_closed': True,
               'last_name': 'Владимирова'},
              {'deactivated': 'deleted',
               'first_name': 'DELETED',
               'id': 3,
               'last_name': ''}]}


In [20]:
# Используя API, определите долю женщин (sex=1) среди пользователей с id от 1 до 500. 
# Иногда будут попадаться пользователи, у которых пол не указан (sex=0), — таких пользователей не нужно учитывать в общем числе.
# В ответе укажите число, округлив до двух знаков после точки-разделителя
ids = ",".join(map(str, range(1, 501)))
params = {'user_ids': ids, 'v': 5.95, 'fields': 'sex', 'access_token': token, 'lang': 'ru'}
dict_of_users = requests.get(url, params=params).json()

have_sex=0
women=0
for i in range(0,500):
    sex = dict_of_users['response'][i]['sex']
    if sex==1:
        women+=1
        have_sex+=1
    if sex==2:
        have_sex+=1
women_share = round(women/have_sex,2)
print(women_share)

0.49


##### **СБОР ИНФОРМАЦИИ ИЗ ГРУПП**
***
